# Se connecter au drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/PLDAC/darknet/
%ls

Mounted at /content/gdrive
/content/gdrive/My Drive/PLDAC/darknet
3rdparty/               image_yolov3.sh        resultats/
backup/                 image_yolov4.sh        results/
build/                  include/               scripts/
build.ps1               json_mjpeg_streams.sh  split_aug.py
cfg/                    LICENSE                src/
cmake/                  Makefile               vcpkg.json
CMakeLists.txt          net_cam_v3.sh          vcpkg.json.opencv23
DarknetConfig.cmake.in  net_cam_v4.sh          video_yolov3.sh
darknet_images.py       obj/                   video_yolov4.sh
darknet.py              prediction/            weights_cfg/
darknet_video.py        prediction_stats/      yolov4.conv.137
data/                   process.py             yolov4.weights
evaluation.py           README.md


# Permettre l'utilisation du GPU

Si vous utilisez colab exécution => modifier le type d'execution => accelerateur materiel => GPU

In [2]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile
!make

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -fPIC -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_old == 0)
      |

# Permettre l'exécution du code

In [3]:
!chmod +x ./darknet
!ls

3rdparty		image_yolov3.sh        resultats
backup			image_yolov4.sh        results
build			include		       scripts
build.ps1		json_mjpeg_streams.sh  split_aug.py
cfg			libdarknet.so	       src
cmake			LICENSE		       uselib
CMakeLists.txt		Makefile	       vcpkg.json
darknet			net_cam_v3.sh	       vcpkg.json.opencv23
DarknetConfig.cmake.in	net_cam_v4.sh	       video_yolov3.sh
darknet_images.py	obj		       video_yolov4.sh
darknet.py		prediction	       weights_cfg
darknet_video.py	prediction_stats       yolov4.conv.137
data			process.py	       yolov4.weights
evaluation.py		README.md


# Réaliser le split

In [4]:
!python process.py

nombre d'images positives: 51
nombre dans train: 44
nombre dans test: 7
others : 0
nombre d'images negatives: 51
nombre dans train: 44
nombre dans test: 7
others : 0


# Ajouter la data augmentation

In [5]:
#!python split_aug.py

# Passer le fichier cfg en mode train

In [6]:
%cd cfg
!sed -i 's/batch=1/batch=32/' yolov4-custom.cfg
!sed -i 's/subdivisions=1/subdivisions=32/' yolov4-custom.cfg
%cd ..

/content/gdrive/My Drive/PLDAC/darknet/cfg
/content/gdrive/My Drive/PLDAC/darknet


# Réaliser le train (enfin) !

In [ ]:
!./darknet detector train data/piford.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 391306, rewritten_bbox = 0.000000 % 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 391306, rewr

# Visualiser la courbe d'apprentissage avec Loss et Map

In [ ]:
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
import matplotlib.pyplot as plt
img = plt.imread('chart.png')
imShow('chart.png')
